In [ ]:
import panel as pn
import holoviews as hv
import geoviews as gv
import cartopy.crs as ccrs
import xmsmesh
import pandas as pd

from earthsim.annotators import PolyAndPointAnnotator
from earthsim.models.custom_tools import CheckpointTool, RestoreTool, ClearTool
import earthsim

hv.extension('bokeh')

%opts Polygons (color='red' alpha=0.5 selection_alpha=0.8 nonselection_alpha=0.2)
%opts Points (size=10 nonselection_alpha=0.5) [tools=['hover']] RGB [width=900 height=600]

In [ ]:
pn.Param.label_formatter.overrides={'status_bar':''}  # https://github.com/pyviz/panel/issues/117

In [ ]:
import param
import xmsmesh
import numpy as np
from shapely.geometry import Polygon, Point
from collections import defaultdict

    
def check_polygon(poly, rtol=1e-05, atol=1e-08):
    """method to fix duplicate start and endpoints"""
    # if the first and last nodes in an array are equal
    if np.allclose(poly[0], poly[-1], rtol, atol, equal_nan=True):
        # strip out the last node and return
        return poly[:-1]
    else:
        # otherwise, return the original poly
        return poly
    
def fix_poly(p):
    boundary = p.exterior
    holes = []
    if boundary.is_ccw:
        boundary.coords = list(boundary.coords)[::-1]
    for h in p.interiors:
        if not h.is_ccw:
            h.coords = list(h.coords)[::-1]
        holes.append(h.coords)
    return Polygon(boundary.coords, holes)

def xmsmesh_to_dataframe(pts, cells):
    """
    Convert mesh pts and cells to dataframe
    
    Args:
      pts (MultiPolyMesherIo.points): Points from a MultiPolyMesherIo
      cells (MultiPolyMesherIo.cells: Cells from a MultiPolyMesherIo
      
    Returns:
      pd.DataFrame: MultiPolyMesherIo points in a dataframe
      pd.DataFrame: MultiPolyMesherIo cells in a dataframe
    """
    r_pts = pd.DataFrame(pts, columns=['x', 'y', 'z'])
    r_cells = pd.DataFrame([(cells[x+2], cells[x+3], cells[x+4]) for x in range(0, len(cells), 5)], columns=['v0', 'v1', 'v2'])
    return r_pts, r_cells

    

In [ ]:
class GenerateMesh(param.Parameterized):
    """
    We need a UUID for each polygon created (https://github.com/pyviz/EarthSim/issues/292) 
    and then we need to apply attirbutes to the UUID. 
    This is an issue right now since polygons are currently not being deleted properly. So they 
    will always be in poly_stream (either as empty list or a single node - I cant remember)
    """
    node_spacing = param.Number(default=1000, bounds=(0,None), softbounds=(10,1000))
    create = param.Action(lambda self: self.create_mesh(),precedence=0.2)
    trigger = param.Boolean(default=True)
    status_bar = param.String(default='') 
    
    
    def __init__(self, **params):
        super(GenerateMesh, self).__init__(**params)
        sample_poly1=dict(
            Longitude = [-10114986, -10123906, -10130333, -10121522, -10129889, -10122959],
            Latitude  = [  3806790,   3812413,   3807530,   3805407,   3798394,   3796693])
        sample_poly2=dict(
            Longitude = [-10095762, -10101582, -10104906],
            Latitude  = [  3804189,   3817180,   3799659])
        sample_points = pd.DataFrame({
            'Longitude': [-10100750],
            'Latitude':  [ 3807009.333],
            'Size':      [   100]})
        
        self.annot = PolyAndPointAnnotator(polys=[sample_poly1, sample_poly2], poly_columns =['Group'], 
            vertex_columns=['Weight'], point_columns=['Size'], points=sample_points) # the only way to get size 
            # from the annotator is specify it going in (https://github.com/pyviz/EarthSim/issues/294)
            
        self.map = self.annot.map_view()
        # self.map.opts.get('style') # merged but not released. (https://github.com/pyviz/holoviews/pull/3440)
        
        self.mesh_io = None
        
        self.comment = []
        
        self.crs = None
        
    def df_to_array(self, df):
        """ This nasty little code converts a df of polygons separated by nans into an array of polygon arrays.
        It should probably be replaced with something more efficient"""
        arr = []
        arr_list = []
        for index, row in df.iterrows():
            if np.isnan(row['Longitude']):
                arr_list.append(arr)
                arr = []
            else:
                arr.append([row['Longitude'], row['Latitude'], 0])

        arr_list.append(arr)
        nparr = np.array([np.array(xi) for xi in arr_list])
        
        return nparr

    def busy(self):
        self.status_bar = "Just wait, I'm thinking..."
    
    def create_mesh(self):
        self.busy()
        
        # Add refine points
        points = self.annot.point_stream.element
#         gv.project(self.annot.point_stream.element, projection=ccrs.GOOGLE_MERCATOR)
        
#         if self.crs: ## todo: never used
#             points = project_points(points, projection=crs)
#         elif 'Size' not in self.annot.point_columns:
#             return []

        refine_points = []
        for x, y, s in zip(*(points.dimension_values(i) for i in range(3))):
            if s:
                refine_points.append(xmsmesh.meshing.RefinePoint(create_mesh_point=True, pt=(x, y, 0), size=float(s)))

            else:
                print('refine point {}, {} skipped due to missing size value'.format(x ,y))

        
        ############ todo: add separate method to individually apply options to the selected polygon
        df = self.annot.poly_stream.element.dframe()

        nparr = self.df_to_array(df)
        
        input_polygon = []
        for data in nparr:
            # instantiate the redistribution class
            rdp = xmsmesh.meshing.PolyRedistributePts()
            # set the node distance
            rdp.set_constant_size_func(self.node_spacing)  # create_constant_size_function
            # run the redistribution function
            outdata = rdp.redistribute(data)
            # AML CHANGE redistribute with the closed loop and then remove the duplicates
            # and reverse the order because the mesher expects clockwise polygons
            # ensure start and end node are different 
            if len(data) != 0:  # todo - waiting on fix for moving poly edges that leave a single node poly behind
                outdata = check_polygon(outdata)[::-1]
            # convert the polygon to an 'input polygon'
            input_polygon.append(xmsmesh.meshing.PolyInput(out_poly=outdata))
            
        # add the input polygons as polygons to the mesher class
        self.mesh_io = xmsmesh.meshing.MultiPolyMesherIo(poly_inputs=input_polygon, refine_points=refine_points)

        # Generate Mesh
        succeded, errors = xmsmesh.meshing.mesh_utils.generate_mesh(mesh_io=self.mesh_io)
        if succeded:
            self.comment.append('Meshing was successful')
        else:
            self.comment.append('Meshing errors found:')
            for err in errors:
                self.comment.append('\t{}'.format(err))

        # convert the xms data format into dataframes
        pts, cells = xmsmesh_to_dataframe(self.mesh_io.points, self.mesh_io.cells)

        # convert the pts df into a hv Points class
        vert_points = gv.Points(pts, vdims=['z'], crs=ccrs.GOOGLE_MERCATOR)
        # create the hv trimesh for displaying unstructured grids
        trimesh = gv.TriMesh((cells, vert_points))
        
        #update the annotator with current drawing - possibly not the best approach (https://github.com/pyviz/EarthSim/issues/293)
        tools = [CheckpointTool(), RestoreTool(), ClearTool()]
        opts = dict(tools=tools, finalize_hooks=[earthsim.annotators.initialize_tools], color_index=None)
        self.annot.polys = self.annot.poly_stream.element.options(**opts)
        self.annot.points = self.annot.point_stream.element.options(**opts)

        # display the edgepaths of the trimesh
        self.map = self.annot.map_view() * trimesh.edgepaths.options(line_width=0.5)  
        # update status bar 
        self.status_bar = self.comment[-1]
        # trigger the redrawing of the map
        self.trigger = True if self.trigger is False else False


    @param.depends('trigger') 
    def view_map(self):
        return self.map
    
    def panel(self):

        return pn.Column(pn.Row(pn.Column(self.view_map, pn.panel(self.param, parameters=['create'], show_name=False)),
                      pn.Tabs(('Input Data', self.annot.table_view()),
                              ('Mesh Options', pn.panel(self.param, parameters=['node_spacing'], show_name=False))
                                )), pn.panel(self.param, parameters=['status_bar'], show_name=False, width=1000))
    

In [ ]:
mesh_panel = GenerateMesh()
mesh_panel.panel()

## Issues to resolve:
+ handle input crs other than ccrs.GOOGLEMERCATOR
+ toolbar is wrong after creating a mesh
+ status bar enhancements

## Lost filigree functionality:
+ clockwise/counter clockwise polygon checking/fixing 
+ repeating first/last node handling
+ handling of holes (filigree._process_polys)
